In [1]:
!pip install conllu
!pip install pyconll
!git clone https://github.com/cpow24/long-doc-coref.git
!pip install transformers==4.2.2

fatal: destination path 'long-doc-coref' already exists and is not an empty directory.


In [12]:
import pandas as pd
import re
import numpy as np
import sys
sys.path.append('long-doc-coref/src')

from google.colab import drive 
drive.mount('/content/gdrive')

import conllu
from io import open
from conllu import parse_tree_incr
import pyconll
from inference.inference import Inference
from  inference.tokenize_doc import *
from transformers import BertTokenizerFast
from  inference.tokenize_doc import *
import torch
import math
import csv

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

In [4]:
"""
Inverse mapping of tokenized text to list text
"""
def inv_map(tokenized_text):
  text = []
  for i in tokenized_text:
    if len(i) < 3:
      text.append(i)
    elif i[0:2] != '##':
      text.append(i)
    else:
      text[-1] = text[-1] + i[2:]
  return text

In [5]:
#Returns cluster_id for given mention
def get_mention_idx(cluster_id, mentioned):
  if mentioned == True:
    val = cluster_id
  else:
    val = 'non-mention' 
  return val

In [6]:
#Recursively extract mention indices and their associated tokens
def extract_mentions(idx, mention_vals, tokens, mention_idx_list, mention_tuple, end_patt):
  mention_idx_list.append(idx)
  mention_tuple += tokens[idx]

  #List potentially containing mention end pattern
  search_vals = re.findall(r'\d+\)', mention_vals[idx])

  #End recursion if end pattern found
  if (end_patt in search_vals):
    return mention_idx_list, mention_tuple
  
  else:
    return extract_mentions(idx + 1, mention_vals, tokens, mention_idx_list, mention_tuple, end_patt)

In [7]:
file_list = ['1023_bleak_house_brat', '105_persuasion_brat', '1064_the_masque_of_the_red_death_brat', '110_tess_of_the_durbervilles_a_pure_woman_brat',
'11231_bartleby_the_scrivener_a_story_of_wallstreet_brat', '113_the_secret_garden_brat', '1155_the_secret_adversary_brat', '11_alices_adventures_in_wonderland_brat',
'1206_the_flying_u_ranch_brat', '120_treasure_island_brat', '1245_night_and_day_brat', '1260_jane_eyre_an_autobiography_brat', '12677_personality_plus_some_experiences_of_emma_mcchesney_and_her_son_jock_brat',
'1327_elizabeth_and_her_german_garden_brat', '1342_pride_and_prejudice_brat', '1400_great_expectations_brat', '145_middlemarch_brat', '15265_the_quest_of_the_silver_fleece_a_novel_brat',
'155_the_moonstone_brat', '158_emma_brat', '160_the_awakening_and_selected_short_stories_brat', '16357_mary_a_fiction_brat', '1661_the_adventures_of_sherlock_holmes_brat',
'1695_the_man_who_was_thursday_a_nightmare_brat', '171_charlotte_temple_brat', '174_the_picture_of_dorian_gray_brat', '18581_adrift_in_new_york_tom_and_florence_braving_the_world_brat', 
'2005_piccadilly_jim_brat', '2084_the_way_of_all_flesh_brat', '208_daisy_miller_a_study_brat', '2095_clotelle_a_tale_of_the_southern_states_brat', '209_the_turn_of_the_screw_brat',
'215_the_call_of_the_wild_brat', '2166_king_solomons_mines_brat', '217_sons_and_lovers_brat', '219_heart_of_darkness_brat', '233_sister_carrie_a_novel_brat', '238_dear_enemy_brat',
'2489_moby_dick_brat', '24_o_pioneers_brat', '2641_a_room_with_a_view_brat', '271_black_beauty_brat', '2775_the_good_soldier_brat', '27_far_from_the_madding_crowd_brat',
'2807_to_have_and_to_hold_brat', '2814_dubliners_brat', '2852_the_hound_of_the_baskervilles_brat', '2891_howards_end_brat', '3268_the_mysteries_of_udolpho_brat', '32_herland_brat',
'33_the_scarlet_letter_brat', '3457_the_man_of_the_forest_brat', '345_dracula_brat', '351_of_human_bondage_brat', '367_country_of_the_pointed_firs_brat', '36_the_war_of_the_worlds_brat',
'4051_lady_bridget_in_the_nevernever_land_a_story_of_australian_life_brat', '41286_miss_marjoribanks_brat', '41_the_legend_of_sleepy_hollow_brat', '4217_a_portrait_of_the_artist_as_a_young_man_brat',
'4276_north_and_south_brat', '4300_ulysses_brat', '432_the_ambassadors_brat', '434_the_circular_staircase_brat', '44_the_song_of_the_lark_brat', '45_anne_of_green_gables_brat',
'472_the_house_behind_the_cedars_brat', '502_desert_gold_brat', '514_little_women_brat', '521_the_life_and_adventures_of_robinson_crusoe_brat', '5230_the_invisible_man_a_grotesque_romance_brat',
'5348_ragged_dick_or_street_life_in_new_york_with_the_bootblacks_brat', '541_the_age_of_innocence_brat', '543_main_street_brat', '550_silas_marner_brat', '599_vanity_fair_brat',
'6053_evelina_or_the_history_of_a_young_ladys_entrance_into_the_world_brat', '60_the_scarlet_pimpernel_brat', '62_a_princess_of_mars_brat', '6593_history_of_tom_jones_a_foundling_brat', 
'711_allan_quatermain_brat', '730_oliver_twist_brat', '73_the_red_badge_of_courage_an_episode_of_the_american_civil_war_brat', '74_the_adventures_of_tom_sawyer_brat',
'766_david_copperfield_brat', '768_wuthering_heights_brat', '76_adventures_of_huckleberry_finn_brat', '77_the_house_of_the_seven_gables_brat', '78_tarzan_of_the_apes_brat',
'805_this_side_of_paradise_brat', '829_gullivers_travels_into_several_remote_nations_of_the_world_brat', '84_frankenstein_or_the_modern_prometheus_brat', '876_life_in_the_ironmills_or_the_korl_woman_brat',
'8867_the_magnificent_ambersons_brat', '932_the_fall_of_the_house_of_usher_brat', '940_the_last_of_the_mohicans_a_narrative_of_1757_brat', '95_the_prisoner_of_zenda_brat',
'969_the_tenant_of_wildfell_hall_brat', '974_the_secret_agent_a_simple_tale_brat', '9830_the_beautiful_and_damned_brat']

In [ ]:
for file in file_list:
  
  #Reading files
  conll_file_str = f'gdrive/My Drive/Colab_Notebooks/coref_extraction/conll/{file}.conll'
  tsv_file_str = f'gdrive/My Drive/Colab_Notebooks/coref_extraction/tsv/{file}.ann'

  conll_df = pd.read_csv(conll_file_str, sep='\t', header=None, quoting=csv.QUOTE_NONE, skiprows=1)
  tsv_df = pd.read_csv(tsv_file_str, sep='\t', header=None)

  #Naming dataframe columns
  conll_df.columns = ['file', 'blank_id', 'sent_id', 'word', 'f1', 'f2','f3', 'f4','f5', 'f6', 'f7', 'f8', 'cluster_id']
  tsv_df.columns = ['cat', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6','mention', 'ment_type','ment_type_2']

  #Retaining only mentions
  tsv_df = tsv_df[tsv_df.iloc[:,0] == 'MENTION']

  #Removing last row as it doesnt contain data
  conll_df.drop(conll_df.tail(1).index,inplace=True)

  #Converting to lower case for string matching
  conll_df['lower'] = conll_df.iloc[:,3].apply(lambda x: x.lower())
  tsv_df['lower'] = tsv_df.iloc[:,6].apply(lambda x: x.lower())

  #Converting text to a tuple of tokens
  tsv_df['tokens'] = tsv_df['lower'].apply(lambda x: tuple(inv_map(flatten(get_tokenized_doc(x, tokenizer)['sentences']))))
  conll_df['tokens'] = conll_df['lower'].apply(lambda x: tuple(inv_map(flatten(get_tokenized_doc(x, tokenizer)['sentences']))))

  #Resetting indices
  tsv_df.reset_index(drop=True, inplace=True)
  conll_df.reset_index(drop=True, inplace=True)

  #Boolean indicator for mentions
  conll_df['mention'] = conll_df.iloc[:,12].notnull()

  #Mention id column
  conll_df['ment_id'] = conll_df.apply(lambda x: get_mention_idx(x.cluster_id, x.mention), axis=1)

  #Extracting a list of mentions and clusters
  mention_ids = []
  mention_locs = []
  mention_tuples = []
  mention_nums = []
  mention_num = -1
  end_patts = []

  for i in range(len(conll_df['tokens'])):
    if conll_df['ment_id'][i] != 'non-mention':

      mention_starts = re.findall(r'\(\d+', conll_df['ment_id'][i])

      for j, start in enumerate(mention_starts):

        mention_num += 1
        mention_nums.append(mention_num)
        mention_id = start[1:]
        end_patt = mention_id + ')'
        mention_ids.append(mention_id)
        mention_loc = []
        mention_tuple = ()
        mention_loc, mention_tuple = extract_mentions(i, conll_df['ment_id'], conll_df['tokens'], mention_loc, mention_tuple, end_patt)
        mention_locs.append(mention_loc)
        mention_tuples.append(mention_tuple)


  #Datframe of mentions
  ment_df = pd.DataFrame(list(zip(mention_nums, mention_ids, mention_tuples, mention_locs)),
                columns =['mention_num', 'mention_id', 'tokens', 'rows'])
  
  #Matching mentions from conll file to mention types from tsv file
  mention_types = []

  for i in range(len(ment_df.index)):
    for j in range(len(tsv_df['lower'])):
      if (ment_df['tokens'][i] == tsv_df['tokens'][j]):
            mention_types.append(tsv_df['ment_type'][j])
            break
      if j == len(tsv_df['lower']) - 1:
        mention_types.append('unknown')
    
  ment_df['type'] = mention_types

  #Boolean indicator for person mentions
  conll_df['person'] = False

  #Marking locations of person mentions as True
  for i, ment_type in enumerate(ment_df['type']):
    if ment_type == 'PER':
      idx = ment_df['rows'][i]
      conll_df['person'][idx] = True

  
  #Removing non-person cluster ids from nested mentions containing a person mention
  for j in range(len(conll_df.index)):
    if conll_df['person'][j] == True:
      for i in range(len(ment_df.index)):
        if ment_df['type'][i] != 'PER':
          full_patt = r"\(\b" + str(ment_df['mention_id'][i]) + r"\b\)"
          start_patt = r"\(\b" + str(ment_df['mention_id'][i]) + r"\b"
          end_patt = r"\b" + str(ment_df['mention_id'][i]) + r"\b\)"

          #Search for full pattern
          if len(re.findall(full_patt, conll_df['ment_id'][j])) > 0:
            conll_df['ment_id'][j] = re.sub(full_patt, '', conll_df['ment_id'][j])
          
          #Search for start pattern
          elif len(re.findall(start_patt, conll_df['ment_id'][j])) > 0:
            conll_df['ment_id'][j] = re.sub(start_patt, '', conll_df['ment_id'][j])

          #Search for end pattern
          elif len(re.findall(end_patt, conll_df['ment_id'][j])) > 0:
            conll_df['ment_id'][j] = re.sub(end_patt, '', conll_df['ment_id'][j])

          else:
            continue
        else:
          continue
    else:
      continue

  #Setting all non-person cluster ids to NaN
  conll_df['new_cluster_id'] = conll_df.apply(lambda x: x['ment_id'] if x['person'] == True else np.nan, axis=1)
  conll_df['new_cluster_id'] = conll_df.apply(lambda x: x['new_cluster_id'] if x['new_cluster_id'] != 'non-mention' else np.nan, axis=1)

  #Ensuring int ids
  conll_df['blank_id'] = conll_df['blank_id'].astype(int)
  conll_df['sent_id'] = conll_df['sent_id'].astype(int)

  #New conll dataframe with only person mentions tagged
  new_conll = conll_df[['file', 'blank_id', 'sent_id', 'word', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8','new_cluster_id']]

  #Exporting conll file
  new_file_str = f'gdrive/My Drive/Colab_Notebooks/coref_extraction/new_conll/{file}.conll'
  new_conll.to_csv(new_file_str, sep ='\t', header=False)